In [1]:
import torch
import os
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data, download_url, extract_zip
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
#load datasets
channel = pd.read_csv('channel_data_final.csv')
video = pd.read_csv('video_data_final.csv')


# rename channelTitle to channelName
video = video.rename({'channelTitle' : 'channelName'}, axis=1)

# drop deprecated columns
channel.drop('Unnamed: 0', inplace=True, axis=1)
video.drop('Unnamed: 0', inplace=True, axis=1)

# create numerical id for all channels
channel['channelId'] = pd.factorize(channel.channelName)[0]
video['channelId'] = pd.factorize(video.channelName)[0]

In [3]:
channel.head()

,channelName,subscribers,views,totalVideos,playlistId,channelId
0,EEG,1740000,1616271673,15332,UUZSJ9bCN0Kh8vJImfacvkFg,0
1,Teleantioquia,511000,121905317,38510,UUeq_SgWZDF8h-ujzwBXI5jQ,1
2,Juan De Montreal,1650000,479065947,1075,UUtUIhr_2EOuLdeAnFYMNnkg,2
3,Katia Nabil,1260000,127790680,391,UUyIE18PSCKxomVDaJ0lEGFw,3
4,Exitosa Noticias,749000,284204440,39447,UUxgO_rak_BKZP8VNVmYqbWg,4


In [4]:
video.head()

,video_id,channelName,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,...,definition,caption,pushblishDayName,durationSecs,tagsCount,likeRatio,commentRatio,titleLength,title_no_stopwords,channelId
0,uCqLtZhf8xw,ElTrollino,¡PASAMOS de TIENDA de MASCOTAS NOOB a TIENDA d...,Vídeo en español de Roblox Tycoon donde pasamo...,['roblox'],2022-07-28T18:15:03Z,819935.0,35640.0,NaN,1343.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,xkQKH6-uNyE,ElTrollino,¡HACKER NOS ENCIERRA EN LA PRISIÓN DE BEDROCK!...,El hacker nos encierra en la prisión más segur...,['minecraft'],2022-06-18T20:45:18Z,3900502.0,112102.0,NaN,2620.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,MWNpBHATT-w,ElTrollino,¡NO ENCUENTRO a LYNA en el LABERINTO! 😱 JUGAMO...,Vlog de Paris y Disney Land donde jugamos al e...,['escondite'],2022-06-10T18:54:53Z,2827316.0,77892.0,NaN,1710.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,6Qg_FpyPY-Y,ElTrollino,¡PASAMOS de GRANJA NOOB a GRANJA PRO en ROBLOX...,Vídeo en español de Farm life Tycoon donde pas...,['roblox'],2022-06-04T18:20:42Z,2692418.0,80655.0,NaN,2659.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,IgNj4zJ-Tv0,ElTrollino,¡Me ESCONDO DENTRO de una MESA DE CRAFTEO! 😂 E...,"El escondite en minecraft con mods, me escondo...",['minecraft'],2022-05-28T18:00:16Z,4171162.0,196737.0,NaN,3183.0,...,hd,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = channel
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [8]:
class SequenceEncoder(object):
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [14]:
class channelEncoder(object):
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        channel = set(g for col in df.values for g in col.split(self.sep))
        mapping = {channel: i for i, video in enumerate(channel)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for channel in col.split(self.sep):
                x[i, mapping[channel]] = 1
        return x

In [16]:
channel_x, channel_mapping = load_node_csv(
    channel, index_col='channelId', encoders={
        'channelName': SequenceEncoder(),
        'channelName': channelEncoder()
    })

TypeError: unhashable type: 'set'